In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import defaultdict

import sqlite3
import sys
import time
import math
import tqdm

from glicko2 import Player

## Loading SQLite Database into Pandas DataFrames

The following code connects to an SQLite database (`melee_player_database.db`) and converts each table within the database into a pandas DataFrame. The DataFrames will be stored in a dictionary, where each key corresponds to the table name with `_df` appended, and the values are the respective DataFrames.

### Steps:

1. **Database Connection**: We use the `sqlite3` library to connect to the SQLite database file.
2. **Retrieve Table Names**: A query retrieves all the table names in the database.
3. **Convert Tables to DataFrames**: For each table:
   - The table is loaded into a pandas DataFrame using `pd.read_sql()`.
   - We check each column to see if any data is JSON-formatted (lists or dictionaries). If so, we convert these columns from strings into their corresponding Python objects using `json.loads()`.
4. **Store DataFrames**: The DataFrames are stored in a dictionary, where the key is the table name with a `_df` suffix, and the value is the DataFrame.
5. **Database Connection Closed**: Once all tables are loaded into DataFrames, the database connection is closed.

### Example:
If the database contains a table named `players`, the corresponding DataFrame will be stored in the dictionary with the key `players_df`, and can be accessed as:

```python
players_df = dfs['players_df']


In [2]:
# Function to get the table names
def get_table_names(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return pd.read_sql(query, conn)['name'].tolist()

# Function to load tables into DataFrames
def load_tables_to_dfs(conn):
    table_names = get_table_names(conn)
    dataframes = {}
    
    for table in table_names:
        # Load table into a DataFrame
        df = pd.read_sql(f"SELECT * FROM {table}", conn)
        
        # Detect and convert JSON formatted columns (if any)
        for col in df.columns:
            # Check if any entry in the column is a valid JSON (list or dictionary)
            if df[col].apply(lambda x: isinstance(x, str)).all():
                try:
                    # Try parsing the column as JSON
                    df[col] = df[col].apply(lambda x: json.loads(x) if pd.notnull(x) else x)
                except (json.JSONDecodeError, TypeError):
                    # If it fails, skip the column
                    pass
        
        # Store the DataFrame with table name + '_df'
        dataframes[f"{table}_df"] = df
        
    return dataframes

# Check if the flag variable exists in the global scope so that this code does not run twice
if 'cell_has_run' not in globals():
    path = "../data/melee_player_database.db"
    
    # Connect to the database
    conn = sqlite3.connect(path)

    # Convert each table into a DataFrame
    dfs = load_tables_to_dfs(conn)

    # Close the connection
    conn.close()

    # Now, you have a dictionary 'dfs' where each key is the table name with '_df' suffix and value is the corresponding DataFrame.
    # For example, to access the DataFrame for a table called 'players':
    # players_df = dfs['players_df']

    dfs['tournament_info_df']['start'] = pd.to_datetime(dfs['tournament_info_df']['start'], unit='s')
    dfs['tournament_info_df']['end'] = pd.to_datetime(dfs['tournament_info_df']['end'], unit='s')

    
    # Set the flag to indicate that the cell has been run
    cell_has_run = True

### Here we adjust the data types of the dataframes so that they are the correct type. (This will be updated as needed.)

In [3]:
dfs['sets_df']['best_of'] = dfs['sets_df']['best_of'].fillna(0).astype(int) 

### Here we make dataframes that we will use and print the head.

The integers in 'characters' count the number of games the player has played that character. (We verify this for Zain below.)

In [ ]:
players_df = dfs['players_df']
players_df.head()

In [ ]:
ranking_df = dfs['ranking_df']
ranking_df.head()

In [ ]:
ranking_seasons_df = dfs['ranking_seasons_df']
ranking_seasons_df.head()

In [ ]:
sets_df = dfs['sets_df']
print(f'{sets_df[sets_df['game_data'].apply(lambda x: len(x) > 0)].shape[0] / sets_df.shape[0]:0.01%} percent of sets have some game data')
sets_df.head()

In [ ]:
tournament_info_df = dfs['tournament_info_df']
tournament_info_df.head()

In [9]:
def process_tournament(tournament_key, player_ratings_df, tournament_info_df, sets_df):
    """
    Process a tournament to update player ratings.
    """
    # Get the sets for this tournament
    tournament_sets_df = sets_df[sets_df['tournament_key'] == tournament_key]
    
    # Extract the unique player IDs from the sets
    tournament_players = pd.unique(tournament_sets_df[['p1_id', 'p2_id']].values.ravel())
    
    # Add any new players to the player_ratings_df
    new_players = [player for player in tournament_players if player not in player_ratings_df.index]
    if new_players:
        new_player_df = pd.DataFrame({
            'dates': [[] for _ in new_players],
            'rating_history': [[] for _ in new_players],
            'rd_history': [[] for _ in new_players],
            'glicko2': [Player() for _ in new_players]
        }, index=new_players)
        player_ratings_df = pd.concat([player_ratings_df, new_player_df], ignore_index=False)
    
    # Ensure the index name is set to 'player_id'
    player_ratings_df.index.name = 'player_id'
    
    # Create a mapping from player_id to their Glicko2 player object
    player_map = player_ratings_df['glicko2'].to_dict()
    
    # Create a snapshot of ratings before the tournament
    ratings_snapshot = player_ratings_df[['glicko2']].copy()
    ratings_snapshot['rating'] = ratings_snapshot['glicko2'].apply(lambda x: x.getRating())
    ratings_snapshot['rd'] = ratings_snapshot['glicko2'].apply(lambda x: x.getRd())
    # Add 'opponent_id' column from the index
    ratings_snapshot.reset_index(inplace=True)
    ratings_snapshot.rename(columns={'player_id': 'opponent_id'}, inplace=True)
    ratings_snapshot = ratings_snapshot[['opponent_id', 'rating', 'rd']]
    
    # Prepare player matches DataFrame
    df_p1 = tournament_sets_df[['p1_id', 'p2_id', 'winner_id']].copy()
    df_p1.rename(columns={'p1_id': 'player_id', 'p2_id': 'opponent_id'}, inplace=True)
    df_p1['outcome'] = (df_p1['winner_id'] == df_p1['player_id']).astype(int)
    df_p1 = df_p1[['player_id', 'opponent_id', 'outcome']]
    
    df_p2 = tournament_sets_df[['p2_id', 'p1_id', 'winner_id']].copy()
    df_p2.rename(columns={'p2_id': 'player_id', 'p1_id': 'opponent_id'}, inplace=True)
    df_p2['outcome'] = (df_p2['winner_id'] == df_p2['player_id']).astype(int)
    df_p2 = df_p2[['player_id', 'opponent_id', 'outcome']]
    
    player_matches = pd.concat([df_p1, df_p2], ignore_index=True)
    
    # Merge to get opponent ratings and RDs
    player_matches = player_matches.merge(
        ratings_snapshot,
        on='opponent_id', how='left'
    )
    player_matches.rename(columns={'rating': 'opponent_rating', 'rd': 'opponent_rd'}, inplace=True)
    
    # Group by player_id to aggregate opponent data
    grouped = player_matches.groupby('player_id').agg({
        'opponent_rating': list,
        'opponent_rd': list,
        'outcome': list
    }).reset_index()
    
    # Get the tournament end date
    end_date = tournament_info_df.loc[tournament_info_df['key'] == tournament_key, 'end'].values[0]
    end_date = pd.to_datetime(end_date)
    
    # Process each player
    for idx, row in grouped.iterrows():
        player_id = row['player_id']
        rating_list = row['opponent_rating']
        rd_list = row['opponent_rd']
        outcome_list = row['outcome']
        
        # Update Glicko rating for the player
        player_glicko = player_map[player_id]
        if rating_list:  # Ensure the player has matches to process
            player_glicko.update_player(rating_list, rd_list, outcome_list)
        
        # Update the player's history
        player_ratings_df.at[player_id, 'dates'].append(end_date)
        player_ratings_df.at[player_id, 'rating_history'].append(player_glicko.getRating())
        player_ratings_df.at[player_id, 'rd_history'].append(player_glicko.getRd())
    
    return player_ratings_df  # Return the updated DataFrame


In [ ]:
sorted_tournament_info_df = tournament_info_df.sort_values('end').reset_index(drop=True)
sorted_tournament_info_df.shape[0]

# Initialize player ratings DataFrame
player_ratings_df = pd.DataFrame(columns=[
     'dates', 'rating_history', 'rd_history', 'glicko2'
])

# Loop over tournaments
for idx, tournament in tqdm.tqdm(sorted_tournament_info_df.iterrows(), total=sorted_tournament_info_df.shape[0]):
    player_ratings_df = process_tournament(tournament['key'], player_ratings_df, tournament_info_df, sets_df)

In [ ]:
df = player_ratings_df.copy()

In [ ]:
# Convert lists to NumPy arrays in the specified columns
for col in ['dates', 'rating_history', 'rd_history']:
    print(col)
    player_ratings_df[col] = player_ratings_df[col].apply(np.array)
# Save the DataFrame to a pickle file
player_ratings_df.to_pickle('../data/overall_players_ranking.pkl')


In [ ]:
# Load the DataFrame to verify
loaded_df = pd.read_pickle('../data/overall_players_ranking.pkl')
loaded_df.head()

In [ ]:
max_rating = 0

for index, row in loaded_df.iterrows():
    index_max_rating = np.argmax(row['rating_history'].astype(int))  # Correcting the typo to `astype`
    if row['rating_history'][index_max_rating] > max_rating:
        best_row = row
        best_index = index
        max_rating = row['rating_history'][index_max_rating]
        print(players_df[players_df['player_id'] == index]['tag'], max_rating)
        # print(max_rating)


        
print(max_rating)


In [ ]:
players_df[players_df['player_id'] == best_index]